In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager
from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0) 

import numpy as np
np.random.seed(0)

In [ ]:
# Set patient and record (selection based on larger number of observations, more seizure events, and consistent channel configurations)

observations = ["chb01_03", "chb01_04", "chb01_15", "chb01_16", "chb01_18", "chb01_21", "chb01_26"]
# observations = ["chb03_01", "chb03_02", "chb03_03", "chb03_04", "chb03_34", "chb03_35", "chb03_36"]
# observations = ["chb10_12", "chb10_20", "chb10_27", "chb10_30", "chb10_31", "chb10_38", "chb10_89"]

# observations = ["PN00_1", "PN00_2", "PN00_4", "PN00_5"]
# observations = ["PN06_2", "PN06_3", "PN06_4", "PN06_5"]
# observations = ["PN14_1", "PN14_2", "PN14_3", "PN14_4"]

observations = ["chb01_03", "chb01_04"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

#### Removing cache data

In [5]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            # Checks if the file begins with "EEG"
            if file.startswith("EEG"):
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

Removed file: .\data\cache\datasets\EEG_CORR-2f1150e5d1acd45b031014443073b786
Removed folder: .\data\cache\oracles\EEG_CORR-2f1150e5d1acd45b031014443073b786


#### Run Gretel experiment

In [6]:
path = 'config/EEG-CORR-topk.jsonc'

context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Evaluating patients: {observations}")
context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
eval_manager = PairedEvaluatorManager(context)

eval_manager.evaluate()

2025-07-02 14:01:40,401 | INFO | 25404 - Evaluating patients: ['chb01_03', 'chb01_04']
2025-07-02 14:01:40,435 | INFO | 25404 - Executing: config/EEG-CORR-topk.jsonc Run: -1
2025-07-02 14:01:40,551 | INFO | 25404 - Creating: EEG_CORR-2f1150e5d1acd45b031014443073b786
2025-07-02 14:01:40,551 | INFO | 25404 - Instantiating: src.dataset.generators.eeg.EEG_CORR
2025-07-02 14:01:43,334 | INFO | 25404 - Saved: EEG_CORR-2f1150e5d1acd45b031014443073b786
2025-07-02 14:01:43,384 | INFO | 25404 - Created: EEG_CORR-2f1150e5d1acd45b031014443073b786
2025-07-02 14:01:43,400 | INFO | 25404 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-07-02 14:01:43,427 | INFO | 25404 - Instantiating: torch.optim.RMSprop
2025-07-02 14:01:43,429 | INFO | 25404 - Instantiating: torch.nn.CrossEntropyLoss
2025-07-02 14:01:43,517 | INFO | 25404 - Creating: OracleTorch-5dcfbb7664b74227753343067ffaa695
2025-07-02 14:01:43,999 | INFO | 25404 - epoch = 0 ---> loss = 0.6838	 accuracy = 0.5104
2025-07-02 14:01:44,333 | IN

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 02/07/2025, ore 14:04


In [9]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [10]:
# # Explainer test indices
# fold_id = eval_manager._evaluators[0]._explainer.fold_id
# print(eval_manager._evaluators[0].dataset.splits[fold_id]['test'])